# Initialization

In [25]:
%reload_ext autoreload
%autoreload 2

In [26]:
from result_saver import SaverProvider

provider = SaverProvider()

# Initialize simulator

In [27]:
from soft_info import RepCodeIQSimulator

DEVICE = 'ibm_sherbrooke'
DISTANCE = 7
ROUNDS = 7
OTHER_DATE = '2023-10-27'
_RESETS = False
LOGICAL = 0 # NOT NEEDED FOR EXTREME IQ BCS HARDCODED 0

_is_hex = True
if DEVICE == 'ibmq_mumbai':
    _is_hex = False

# Initialize simulator
simulator = RepCodeIQSimulator(provider, DISTANCE, ROUNDS, DEVICE, _is_hex=_is_hex, _resets = _RESETS, other_date=OTHER_DATE)

# Dev get IQ data

In [28]:
import numpy as np

SHOTS = int(1e3)
NOISE_LIST = [3e-2, 0.8e-2, 1e-2, 3e-2] # [two-qubit-fidelity, reset error, measurement error, idle error]


stim_circuit = simulator.get_stim_circuit(NOISE_LIST)
meas_outcomes = stim_circuit.compile_sampler(seed=42).sample(SHOTS)
print("generated counts")

if not simulator._resets:
    # LOGIC
    pass

IQ_memory = np.zeros_like(meas_outcomes, dtype=np.complex128)
len_IQ_array = len(simulator.qubit_mapping)
kde_samples_needed = {qubit_idx: {'0': 0, '1': 0} for qubit_idx in simulator.kde_dict.keys()}
sample_counters = {qubit_idx: {'0': 0, '1': 0} for qubit_idx in simulator.kde_dict.keys()}    

print("starting kde samples needed")
for row in meas_outcomes:
    for IQ_idx, bit in enumerate(row):
        qubit_idx = simulator.qubit_mapping[IQ_idx]
        kde_samples_needed[qubit_idx][str(int(bit))] += 1

print("starting kde samples")
kde_samples = {}
for qubit_idx, needed_nb_samples in kde_samples_needed.items():
    [kde0, kde1], scaler = simulator.kde_dict[qubit_idx], simulator.scaler_dict[qubit_idx]
    if needed_nb_samples['0'] > 0:
        samples0 = scaler.inverse_transform(kde0.sample(needed_nb_samples['0'], random_state=42))
    else:
        samples0 = np.empty((0, 2)) 
    if needed_nb_samples['1'] > 0:
        samples1 = scaler.inverse_transform(kde1.sample(needed_nb_samples['1'], random_state=42))
    else:
        samples1 = np.empty((0, 2)) 
    kde_samples[qubit_idx] = {'0': samples0, '1': samples1}

print("starting IQ memory")
for row_idx, row in enumerate(meas_outcomes):
    for IQ_idx, bit in enumerate(row):
        qubit_idx = simulator.qubit_mapping[IQ_idx]
        sample_index = sample_counters[qubit_idx][str(int(bit))]
        sample = kde_samples[qubit_idx][str(int(bit))][sample_index]   
        IQ_memory[row_idx, IQ_idx] = complex(sample[0], sample[1])
        sample_counters[qubit_idx][str(int(bit))] += 1

assert sample_counters == kde_samples_needed

print(IQ_memory.shape)
IQ_memory

generated counts
starting kde samples needed
starting kde samples
starting IQ memory
(1000, 49)


array([[-1.50232187e+07+2.25052950e+07j, -1.03235821e+08+5.94349540e+07j,
        -9.51286138e+07-7.84326738e+07j, ...,
         2.95112146e+07-3.72114071e+07j, -1.75257103e+07-1.82129012e+07j,
        -3.22184177e+07+7.50352684e+07j],
       [-1.19983569e+06+1.54127207e+07j, -5.04871769e+07+7.20190162e+07j,
        -4.66470942e+07-7.41325525e+07j, ...,
         2.00817472e+07-3.17737481e+07j,  1.33709326e+07-2.99407897e+07j,
        -6.60751643e+07+6.26890487e+07j],
       [-1.64742047e+07+2.11673800e+07j, -7.17076868e+07+5.90467004e+07j,
        -7.85188060e+07-7.33950965e+07j, ...,
        -2.05160111e+07-3.31793620e+07j, -2.74901095e+07-2.05009485e+07j,
        -6.13858592e+07+6.21946651e+07j],
       ...,
       [-1.30162782e+07+2.33196434e+07j, -3.50650937e+07+7.45803429e+07j,
        -8.31946968e+07-1.05510371e+08j, ...,
        -1.80775896e+07-2.25086711e+07j, -3.08294325e+07-1.77899449e+07j,
        -4.65463895e+07+7.54711614e+07j],
       [-8.18643514e+06+1.80830190e+07j, -6.

In [42]:
import numpy as np

SHOTS = int(1e5)
NOISE_LIST = [3e-2, 0.8e-2, 1e-2, 3e-2] # [two-qubit-fidelity, reset error, measurement error, idle error]


stim_circuit = simulator.get_stim_circuit(NOISE_LIST)
meas_outcomes = stim_circuit.compile_sampler(seed=42).sample(SHOTS)
print("generated counts")

if not simulator._resets:
    # LOGIC
    pass

IQ_memory = np.zeros_like(meas_outcomes, dtype=np.complex128)
# print(f"IQ_memory.shape: {IQ_memory.shape}")
# CORRECT 


len_IQ_array = len(simulator.qubit_mapping)
# print(f"len_IQ_array: {len_IQ_array}")
# CORRECT

used_qubits = set(simulator.qubit_mapping.values())
kde_samples_needed = {qubit_idx: {'0': 0, '1': 0} for qubit_idx in simulator.kde_dict.keys()}
sample_counters = {qubit_idx: {'0': 0, '1': 0} for qubit_idx in simulator.kde_dict.keys()}    
# print("qubit_mapping: ", simulator.qubit_mapping)  
# CORRECT



print("starting kde samples needed")
for IQ_idx in range(len_IQ_array):
    qubit_idx = simulator.qubit_mapping[IQ_idx]  # Find the corresponding qubit index
    # Get the binary outcomes for this qubit across all shots
    outcomes = meas_outcomes[:, IQ_idx]
    # Use np.bincount to count '0's and '1's. Ensure minlength=2 for the case all outcomes are the same.
    counts = np.bincount(outcomes, minlength=2)
    # print(f"counts: {counts} at IQ_idx: {IQ_idx} and qubit_idx: {qubit_idx}")
    # Now, update kde_samples_needed directly with these counts.
    kde_samples_needed[qubit_idx]['0'] += counts[0]
    kde_samples_needed[qubit_idx]['1'] += counts[1]

# print(f"kde_samples_needed: {kde_samples_needed}")

# # add up the samples needed for each qubit
# total = 0
# for k,v in kde_samples_needed.items():
#      total += v['0'] + v['1']

# print(f"total: {total}")



print("starting kde samples")
kde_samples = {}
for qubit_idx, needed_nb_samples in kde_samples_needed.items():
    [kde0, kde1], scaler = simulator.kde_dict[qubit_idx], simulator.scaler_dict[qubit_idx]
    if needed_nb_samples['0'] > 0:
        samples0 = scaler.inverse_transform(kde0.sample(needed_nb_samples['0'], random_state=42))
    else:
        samples0 = np.empty((0, 2)) 
    if needed_nb_samples['1'] > 0:
        samples1 = scaler.inverse_transform(kde1.sample(needed_nb_samples['1'], random_state=42))
    else:
        samples1 = np.empty((0, 2)) 
    kde_samples[qubit_idx] = {'0': samples0, '1': samples1}

print("starting IQ memory")
next_sample_indices = np.zeros((len(simulator.kde_dict), 2), dtype=int)

# Iterate over meas_outcomes to fill IQ_memory
for row_idx, row in enumerate(meas_outcomes):
    for IQ_idx, bit in enumerate(row):
        qubit_idx = simulator.qubit_mapping[IQ_idx]
        bit_int = int(bit)
        
        # Get the next sample index for this qubit and bit
        sample_index = next_sample_indices[qubit_idx, bit_int]
        
        # Retrieve the sample and update IQ_memory
        sample = kde_samples[qubit_idx][str(bit_int)][sample_index]
        IQ_memory[row_idx, IQ_idx] = complex(sample[0], sample[1])
        
        # Increment the sample index for the next use
        next_sample_indices[qubit_idx, bit_int] += 1

print(IQ_memory.shape)
IQ_memory

generated counts
starting kde samples needed
starting kde samples
starting IQ memory
(100000, 49)


array([[-12625257.20513788+2.39492004e+07j,
         24186833.62788546+7.66390138e+07j,
        -52886891.85341469-7.75103778e+07j, ...,
        -23357486.65135763-3.14976252e+07j,
        -23564967.91028597-2.06558989e+07j,
        -53987524.82207315+7.38601625e+07j],
       [-24188830.70587259+2.57560592e+07j,
        -57942525.7807887 +7.26142978e+07j,
        -81952710.03958131-1.20597476e+08j, ...,
         17147192.95281468-3.51548997e+07j,
        -20711192.38156219-2.11781353e+07j,
        -63113631.9239763 +6.25539646e+07j],
       [-14989248.62669082+1.84325645e+07j,
        -61646914.3020632 +6.31231368e+07j,
        -96907597.86149655-9.12244833e+07j, ...,
         22352364.41691762-2.79001361e+07j,
         12272415.98473733-3.24092084e+07j,
        -83818028.6817148 +6.05005086e+07j],
       ...,
       [-10482074.64167522+1.41710368e+07j,
        -45003870.46757969+7.81546774e+07j,
        -85519897.81026025-8.48422375e+07j, ...,
        -30471063.56802413-1.83424566e+07

In [36]:
IQ_data = simulator.generate_IQ(SHOTS, NOISE_LIST)
print(IQ_data.shape)
IQ_data

generated counts
correcting counts
finished correcting counts
(100000, 49)


array([[-1.54573238e+07+21958401.55828892j,
         4.10053419e+07+75995596.9591296j ,
        -6.06395759e+07-72425347.53439532j, ...,
        -2.33574867e+07-31497625.24369372j,
        -2.35649679e+07-20655898.92745575j,
        -5.39875248e+07+73860162.48295219j],
       [-2.38026700e+07+25287198.4659813j ,
        -1.00799677e+08+64036703.99002467j,
        -4.73661056e+07-65075859.19687505j, ...,
         1.71471930e+07-35154899.69432529j,
        -2.07111924e+07-21178135.34771013j,
        -6.31136319e+07+62553964.56688988j],
       [-1.27586701e+07+17223341.25832713j,
        -3.07107899e+07+58781514.13314077j,
        -8.21088036e+07-92875613.41956146j, ...,
         2.23523644e+07-27900136.0509418j ,
         1.22724160e+07-32409208.38731221j,
        -8.38180287e+07+60500508.56824557j],
       ...,
       [-9.34108760e+06+20136096.89514977j,
        -4.71502959e+07+69105299.11473085j,
        -1.02926159e+08-61682721.65112722j, ...,
        -2.63755046e+07-20836701.24843341

# Get IQ data

In [41]:
IQ_data = simulator.generate_IQ(SHOTS, noise_list=NOISE_LIST)

generated counts
correcting counts
finished correcting counts
